# Polynomial acclerated Gibbs sampling

* 조원: 김동욱, 김민우, 김민지
* Paper: Fox Colin, and Albert Parker. “Accelerated Gibbs sampling of normal distributions using matrix splittings and polynomials.” Bernoulli 23.4B (2017): 3711-3743.

---

## 1. Introduction

* Our problem is sampling from normal distirubions.
* Gibbs sampler is commonly used because of simple implementaion 
* However, the Gibbs sampler is not efficient for a massive models.
* C. Fox and A.Parker(2017) proposed a generalized and accelerated Gibbs samplers which is fast for high-dimensional normal distributions.
* In this presentation, we review the paper of C. Fox and A. Parker(2017) "Accelerated Gibbs sampling of normal distributions using matrix splittings and polynomials" -  Bernoulli 23.4B and show the reseult of the proposed algorithm re-written by **Julia**.

---

## 2. Problem setting

#### Problem
1.  We want to compute a sample
   $$
   \mathbf{y} \sim \mathbf{N}(0, A^{-1})
   $$
   , where $A$ is a given precision matrix. 
2.  An iterative samplers, such as Gibbs, are good option when the dimension is high becuase of it's inexpensive cost per itertion and small computer memroy requirements. 
3.  If the precision matrix $A$ is sparse with $\mathcal{O}(n)$ non-zero elements, iterative methods cost only about $2n$ flops per iteration.
4.  So, our goal is find an **Iterative method which converges in small iterations, for example, significantly less than $\mathcal{O}(n^2)$ iterations**.



#### Non-iterative sampler: Cholesky factorization

<center><img src="img/algorithm1.png"></center>

* This is a baseline algorithm and non-iterative. 
* Cholesky factorizing requires that the precision matrix and the Cholesky factor be stored in computer memory, which can be prohibitive for large problems.

#### Gibbs sampling from a Normal Distribution

<center><img src="img/algorithm2.png"></center>

* The iteration can be written in the matrix form,
$$
    \mathbf{y}^{(k+1)} = -\mathbf{D}^{-1}\mathbf{L}\mathbf{y}^{(k+1)} - \mathbf{D}^{-1}\mathbf{L}^T\mathbf{y}^{(k)} + \mathbf{D}^{-1/2}\mathbf{z}^{(k)}
    \tag{1}
$$
, where $\mathbf{z}^{(k)} \sim N(0, \mathbf{I})$, $\mathbf{D} = \text{diag}(\mathbf{A})$, and 
$\mathbf{L}$ is the strictly lower triangluar part.

### Splitting Iterative Methods
* The iterative method for solving $\mathbf{A}\mathbf{x}=\mathbf{b}$ split the matrix 
$\mathbf{A} = \mathbf{M} - \mathbf{N}$, where $\mathbf{M}$ is invertible and easy to invert.

* Then $\mathbf{A}\mathbf{x} = \mathbf{M}\mathbf{x} - \mathbf{N}\mathbf{x} = \mathbf{b}$ or
    $$
        \mathbf{x} = \mathbf{M}^{-1}\mathbf{N}\mathbf{x} - \mathbf{M}^{-1}\mathbf{b}.
    $$

* Thus a solution to $\mathbf{A}\mathbf{x}=\mathbf{b}$ is a fixed point of iteration
    $$
        \mathbf{x}^{(k+1)} = \mathbf{M}^{-1}\mathbf{N}\mathbf{x}^{(k)} - \mathbf{M}^{-1}\mathbf{b}. \tag{2}
    $$
* The iterative solution is convergent(i.e., $\mathbf{x}^{(k)} \rightarrow \mathbf{A}^{-1}\mathbf{b}$)
if and only if 
$$\mathcal{\rho}(\mathbf{M}^{-1}\mathbf{N}) < 1$$ 
,where $\mathcal{\rho}(\cdot)$ is the spectral radius of a matrix. 
* Let the error at step $k$ be $\mathbf{e}^{(k+1)} =\mathbf{x}^{(k+1)} - \mathbf{A}^{-1}\mathbf{b}.$
Then it's well known(e.g. Axelsson) that
$$
    \lim_{k \to \infty}
    \bigg(\frac{\|\mathbf{e}^{(k+1)}\|_2}{\|\mathbf{e}^{(0)}\|_2}\bigg)^{1/k}  
    = \mathcal{\rho}(\mathbf{M}^{-1}\mathbf{N})
    \tag{3}
$$

### Splitting iterative linear solvers

* For a symmetric matrix $\mathbf{A}$, let $\mathbf{A} = \mathbf{L} + \mathbf{D} + \mathbf{L}^T$
, where $\mathbf{L}$ is a strictly lower triangular part and $\mathbf{D}$ is a diagonal part.

|Splitting|$\mathbf{M}$|Convergence|
|-------------|--------------|:-----------:|
|Richardson|$\frac{1}{\omega}\mathbf{I}$|$0 < \omega < \frac{2}{\rho(\mathbf{A})}$|
|Jacobi|$\mathbf{D}$|$\mathbf{A}$ strictly diagonally dominant|
|Gauss-Seidel(GS)|$\mathbf{D} + \mathbf{L}$|always|
|SOR|$\frac{1}{\omega}\mathbf{D} + \mathbf{L}$|$0 < \omega < 2$|
|SSOR|$\frac{\omega}{2 - \omega}\mathbf{M}_{\text{SOR}}\mathbf{D}^{-1}\mathbf{M}_{\text{SOR}}^T$|$0 < \omega < 2$|

* For the Gauss-Seidel method, (2) becomes

    $$
        \mathbf{x}^{(k+1)} = -\mathbf{D}^{-1}\mathbf{L}\mathbf{x}^{(k+1)} - \mathbf{D}^{-1}\mathbf{L}^T\mathbf{x}^{(k)} + \mathbf{D}^{-1}\mathbf{b}. \tag{4}
    $$
    
    Note that the gibbs sampler iteration (1) and linear solver's (4) are almost same. 

---

## 3. Equivalence between iterative linear solvers and Gibbs samplers

### Equivalence of linear solvers and Gibbs samplers
##### Theorem 1. (Equivalence of iterative linear solvers and Gibbs samplers)

>Let $\mathbf{A} = \mathbf{M} - \mathbf{N}$ be a splitting with $\mathbf{M}$ invertible, and let
$\pi(\cdot)$ be some fixed probability distribution with zero mean and fixed non-zero covariance.
For any fixed vector $\mathbf{b}$, and random vectors 
$\mathbf{c}^{(k)} \overset{\text{i.i.d.}}{\sim} \pi$, $k = 0, 1, 2, \dots,$ 
the stationary linear iteration
$$
    \mathbf{x}^{(k+1)} = \mathbf{M}^{-1}\mathbf{N}\mathbf{x}^{(k)} + \mathbf{M}^{-1}\mathbf{b}
$$
converges, with $\mathbf{x}^{(k)} \to \mathbf{A}^{-1}\mathbf{b}$ as $k \to \infty$ whatever
the initial vector $\mathbf{x}^{(0)}$, if and only if there exists a distribution $\Pi$ such that
the stochastic iteration
$$
    \mathbf{y}^{(k+1)} = \mathbf{M}^{-1}\mathbf{N}\mathbf{y}^{(k)} + \mathbf{M}^{-1}\mathbf{c}^{(k)} \tag{5}
$$
converges in distribution on $\Pi$, with $\mathbf{y}^{(k)} \overset{\mathcal{D}}{\to} \Pi$ as
$k \to \infty$ whatever the initial state $\mathbf{y}^{(0)}$.



### Sampling from normal distributions using matrix splittings
##### Theorem 2. (Convergence of first and second moments)

>Let $\mathbf{A}$ be SPD, $\mathbf{A} = \mathbf{M} - \mathbf{N}$ be a convergent splitting,
$\mathbf{\mu}$ a fixed vector, and $\pi(\cdot)$ a fixed probability distribution with finite mean
$\mathbf{\nu}$ and non-zero covariance $\mathbf{V}$. Consider the stochastic iteration (5) where
$\mathbf{c}^{(k)} \overset{\text{i.i.d.}}{\to}\pi$, $k = 0, 1, 2, \dots .$ Then, whatever the starting
state $\mathbf{y}^{(0)}$, the followings are equivalent:
* $\textbf{E}[\mathbf{c}^{(k)}] = \mathbf{\nu}$ and 
$\textbf{Var}(\mathbf{c}^{(k)}) = \mathbf{V} = \mathbf{M}^T + \mathbf{N}$;
* the iterates $\mathbf{y}^{(k)}$ converges in distribution to some distribution $\Pi$ that
has mean $\mathbf{\mu} = \mathbf{A}^{-1}\mathbf{\nu}$ and covariance matrix $\mathbf{A}^{-1}$;
in particular $\textbf{E}[\mathbf{y}^{(k)}] \to \mathbf{\mu}$ and 
$\textbf{Var}(\mathbf{y}^{(k)}) \to \mathbf{A}^{-1}$ as $k \to \infty$.


* This theorem shows blue **how to design** the noise distribution $\pi$ so that the limit distribution $\Pi$ has a desired mean $\mu$ and covariance $\Sigma = \mathbf{A}^{-1}$.
* If we set $\pi = \text{N}(\mathbf{\nu}, \mathbf{V})$ then the following are equivalent:
    1. $\mathbf{V} = \mathbf{M}^T + \mathbf{N}$; 
    2. $\mathbf{y}^{(k)} \overset{\mathcal{D}}{\to} \mathbf{N}(\mathbf{\mu}, \mathbf{A}^{-1})$
    , where $\mathbf{\mu} = \mathbf{A}^{-1}\mathbf{\nu}$.

* The followings are algorithms of Iterative Samplers and SSOR sampler. Note that for the SSOR sampler, $M_{\text{SOR}}$ calculated implicitly.

 Iterative samper |  SSOR sampler
:-------------------------:|:-------------------------:
![](img/algorithm3.png)  |  ![](img/algorithm4.png)

---

## 4. Proposed methods and Theory

### Accleration of linear solvers by polynomials
* Most of the results in this section follows from the [Axelsson, Iterative Solution Methods](https://www.cambridge.org/core/books/iterative-solution-methods/934B9797ADF03EBC377B20622DADE822)
* For a set of *acceleration parameters* $\{ \{\alpha_k\}, \{\tau_k\}\}$, let's introduce the second order iteration
    $$ \mathbf{x}^{(k+1)} = (1-\alpha_k)\mathbf{x}^{(k-1)} + \alpha_k \mathbf{x}^{(k)} +
       \alpha_k\tau_k\mathbf{M}^{-1}(\mathbf{b} - \mathbf{A}\mathbf{x}^{(k)}). \tag{6}
    $$

* At the first step, $\alpha_0 = 1$ and $\mathbf{x}^{(1)} = \mathbf{x}^{(0)} + \tau_0\mathbf{M}^{-1}(\mathbf{b} - \mathbf{A}\mathbf{x}^{(0)})$.
* Next, generate a (k+1)st order polynomial $P_{k+1}$ recursively as
    $$
    P_{k+1}(\lambda) = (\alpha_k - \alpha_k\tau_k\lambda)P_k(\lambda) + (1-\alpha_k)P_{k-1}(\lambda). \tag{7}
    $$

* Then the $k$-step error $\mathbf{e}^{(k)} = \mathbf{x}^{(k)} - \mathbf{A}^{-1}\mathbf{b}$ may be written as 
    $$
    \mathbf{e}^{(k+1)} = P_k(\mathbf{M}^{-1}\mathbf{A}) \mathbf{e}^{(0)}  \tag{8}
    $$
  , which can be compared directly to (3).


* When estimates of the extreme eigenvalues $\lambda_{\min}$, $\lambda_{\max}$ of 
$\mathbf{M}^{-1}\mathbf{A}$ are available($\lambda_{\min}$, $\lambda_{\max}$ are real when
$\mathbf{M}, \mathbf{N}$ are symmetric), then following coefficients
$\{ \{\alpha_k\}, \{\tau_k\}\}$ generate the scaled **Chebyshev polynomials**

$$  \tau_k = \frac{2}{\lambda_{\max} + \lambda_{\min}}, \quad
    \beta_k = \bigg( \frac{1}{\tau_k} - \beta_{k-1}
    \bigg( \frac{\lambda_{\max} - \lambda_{\min}}{4}
    \bigg)^2 \bigg)^{-1}, \quad
    \alpha_k = \frac{\beta_k}{\tau_k},
$$

where $\alpha_0 = 1$ and $\beta_0 = \tau_0$.

Remarks:
>* We denote the $k$-th order Chebyshev polynomial as $\mathcal{Q}_k$.
>*  Note that these parameters are independent of the iterates $\{\mathbf{x}^{(k)}\}$.
>* $\mathbf{M}$ is required to be symmetric, applying Chebyshev acceleration to SSOR is a common paring.


* From *Axelsson*,

$$  \mathcal{Q}_k = \underset{{P_k \in \mathbb{P}_k}}{\text{argmin}}
    \bigg(
    \underset{\lambda \in [\lambda_{\min}, \lambda_{\max}]}{\max} |P_k(\lambda)|
    \bigg),
$$

where $\mathbb{P}_k$ is a space of $k$th order polynomials. The optimal value is

$$ \underset{\lambda \in [\lambda_{\min}, \lambda_{\max}]}{\max} |\mathcal{Q}_k(\lambda)|
    = \frac{2\sigma^k}{1 + \sigma^{2k}}, \quad
    \text{where }
    \sigma = \frac{1 - \sqrt{\lambda_{\min}/\lambda_{\max}}}
    {1 + \sqrt{\lambda_{\min}/\lambda_{\max}}} \in [0, 1).
$$

From (8), 
$\mathbf{e}^{(k+1)} = \mathcal{Q}_k(\mathbf{M}^{-1}\mathbf{A})\mathbf{e}^{(0)}$ and
then the asymptotic convergence factor is bounded above by

$$ \lim_{k \to \infty} \{\max |\mathcal{Q}_k(\lambda)|\}^{1/k} = \sigma .$$

* *Axelsson* also shows the convergence factor of non-accelerated iterative solver is
bounded below by $$\rho = \frac{1 - \lambda_{\min}/\lambda_{\max}}{1 + \lambda_{\min}/\lambda_{\max}}.$$
Then $\sigma < \rho$ always holds, so the Chebyshev acceleration **really accelerate** a calculation procedures.

### Acceleration of Gibbs sampling by polynomials

* By slightly chainging (6), we can consider the second order stochastic iteration
    $$  \mathbf{y}^{(k+1)} = 
        (1-\alpha_k)\mathbf{y}^{(k-1)} + \alpha_k \mathbf{y}^{(k)} +
        \alpha_k\tau_k\mathbf{M}^{-1}(\mathbf{c}^{(k)} - \mathbf{A}\mathbf{y}^{(k)}). \tag{9}
    $$
    
    Only diffrenet thing is the vector $\mathbf{b}$ 
    has been replaced by a random vector $\mathbf{c}^{(k)}$.
* Next, introduce some coefficients $a_k, b_k, \kappa_k$ defined by
    $a_k = (2-\tau_k)/\tau_k + (b_k-1)(1/\tau_k + 1/\kappa_k - 1)$, 
    $b_k = 2\kappa_k(1-\alpha_k)/(\alpha_k \tau_k) + 1$, 
    $\kappa_{k+1} = \alpha_k \tau_k + (1-\alpha_k)\kappa_k$, and $\kappa_1 = \tau_0$.

* Also, suppose that $\{\{\alpha_k\}, \{\tau_k\}\}$ that are independent of 
    $\{\mathbf{x}^{(k)}\}$.
* Then (9) converges in distribution to our targe distribution if the polynomial accerated linear solver converges.



### Main Results
##### Theorem 3. (Accelerated Gibbs sampler)
>Let $\mathbf{A}$ be SPD and $\mathbf{A} = \mathbf{M} - \mathbf{N}$ be a symmetric splitting. Define an noise vectors 
$\mathbf{c}^{(k)} \overset{ind.}{\sim} (\mathbf{\nu}, a_k \mathbf{M} + b_k \mathbf{N})$.
>1. If the accelerated linear solver (7) converges to $\mathbf{A}^{-1}\mathbf{b}$
    then the accelerated stochastic iteration (14)
    converges to a distribution with moments $(\mathbf{A}^{-1}\nu, \mathbf{A}^{-1}).$ 
    Furthermore, if the $\{\mathbf{c}^{(k)}\}$ are normal, then
    $$  \mathbf{y}^{(k)} \overset{\mathcal{D}}{\to} 
        \mathbf{N}(\mathbf{\mu} = \mathbf{A}^{-1}\nu, \mathbf{A}^{-1}).$$
2. $\mathbf{E}(\mathbf{y}^{(k)})-\mathbf{A}^{-1}\nu 
= P_k(\mathbf{M}^{-1}\mathbf{A})(\mathbf{E}(\mathbf{y}^{(0)})-\mathbf{A}^{-1}\nu)\to 0$
3. $\mathbf{Var}(\mathbf{y}^{(k)})-\mathbf{A}^{-1} 
= P_k(\mathbf{M}^{-1}\mathbf{A})(\mathbf{Var}(\mathbf{y}^{(0)})-\mathbf{A}^{-1})\to 0$



* Chebyshev polynomial accelerated normal sampler is guaranteed to
converge faster than any other acceleration scheme that has the parameters 
$\{ \alpha_k, \tau_k \}$ independent of the iterates $\{\mathbf{y}^{(k)}\}$.



---

## 5. Computed examples


### Section 6.1 in the paper

#### 1. Data

* A *first order locally linear* sparse precision matrix $\mathbf{A} \in \mathbb{R}^{n^2 \times n^2}$ is defined by

$$
\mathbf{A}_{ij}=
10^{-4}\delta_{ij} + 
\begin{cases}
n_i, \quad &\text{if } i = j, \\
-1,\quad &\text{if } i \neq j \text{ and } \|s_i - s_j\|_2 \leq 1, \\
0 \quad &\text{otherwise.}
\end{cases}
$$

* The discrete points $\{s_i\}$ are on a regular $n \times n$ lattice over the two dimensional domain $\mathcal S = [1, n] \times [1, n]$.
  So, $s_i$ is a point of $(j, k)$ where $i = j + n \cdot (k-1)$. For example, $s_1 = (1, 1)$, $s_{10} = (10, 1)$ and $s_{23} = (3, 3)$ when $n = 10$.

* We use a sparse structure to implement. Note that a size of A increases very fast for the terms of $n^4$. For the detail implementation, see `"src/matrixgen.jl"`. With our code, $A$ matrix with $n=512$ can also be handled, however dense matrix of size $512^4 = 2^{36}$ causes a `out of memory()` problem.

* The following plot is a sparsity pattern of $\mathbf{A}$ when $n = 10$.

In [2]:
include("../src/matrixgen.jl")
import UnicodePlots: spy
A = laplacematrix(10)
spy(A)

                    Sparsity Pattern
       ┌──────────────────────────────────────────┐    
     1 │⠻⣦⡀⠀⠘⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ > 0
       │⠀⠈⠻⣦⡀⠀⠉⢆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ < 0
       │⠒⢄⠀⠈⠱⣦⡀⠀⠑⢢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠣⢄⠀⠈⠻⣦⡀⠀⠑⠤⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠱⣀⠀⠈⠱⣦⡀⠀⠑⢄⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠑⡄⠀⠈⠻⣦⡀⠀⠘⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠈⠑⢄⠀⠈⠱⣦⡀⠀⠉⢆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠒⢄⠀⠈⠻⣦⡀⠀⠑⢢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠣⢄⠀⠈⠛⣤⡀⠀⠑⠤⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠱⣀⠀⠈⠻⣦⡀⠀⠑⢄⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⡄⠀⠈⠛⣤⡀⠀⠘⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠑⢄⠀⠈⠻⣦⡀⠀⠉⢆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠒⢄⠀⠈⠛⣤⡀⠀⠑⢢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠣⢄⠀⠈⠻⣦⡀⠀⠑⠤⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠱⣀⠀⠈⠻⢆⡀⠀⠑⢄⡀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⡄⠀⠈⠻⣦⡀⠀⠘⢄⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀

#### 2. Algorithm implementation
* Our major purpose is implementation of `Cheby-acclerated sampling`. It's detail algorithm given in the supplementary of the paper. See `algorithm-chebysampler.png` in the current directory. Also, we implements a lot of Iterative Solvers and related samplers follow the flow of a SSOR sampler algorithm.

* Writing down direct mathmatical expression raises **huge time delay and memory allocation problem**, Even few attemptions for reduction memory(Sparse, using basic matrix inversion operations) were failed. `k3_ssor` is what we implemented and it's performance is poor relative to `ssor` from `IterativeSolvers.jl`.

|Example of Direct implementation of the Algorithm, Algorithm3  |  Benchmark results|
|:-------------------------:|:-------------------------:|
|![](img/proto_performance_3.png)  |  ![](img/proto_performance.png)|

##### Algorithm structure design
We benchmark Julia itterative solver algorithms and figure out how to reduce computing time and memory allocations.  

* The key for reducing time and memory is to declare all variables used in the update process in advance so that there is no additional memory allocation for them, define the new structure to access to the precision matrix for subsequent operations and implement all operations elementwisely or using forward and backward computations.
* For this purpose, we follow 'IterativeSolver.jl''s *DiagonalIndices structure* and sparse based inplacement operation.
* In julia sparse package, [SparseMatricCSC](https://github.com/JuliaLang/julia/blob/46ce4d79337bdd257ee2e3d2f4bb1c55ff0a5030/stdlib/SparseArrays/src/sparsematrix.jl#L11-L18) type is only storage 3 types of vectors; colum pointers, none zero values and row values
* Itterative solver use the design Diagonal structure and store one more array ; Diagonal indices of none zero values.
* Using Diagonal structure, Forward and backward operation can be computed without declaring new Lower or Upper Triangluar matrix.
* For the detail implementation, see `"src/cheby.jl"`, `"src/cheby_sampler.jl"`.
* Below figures are performance comparison between itteration solvers from `IterativeSolvers.jl` vs `implemented solvers`. Our algorithms have more advantage in saving memory allocations over 10 times compared with itterative solver and also faster than Julia itteration solver package.
    
 |SOR |  SSOR|
|:-------------------------:|:-------------------------:|
|![](img/compare_performance_2.png)  |  ![](img/compare_performance.png)|

#### 3. Implemented Accelerated Solvers.

* We implement a Iterative Solvers: `Gauss-Seidel`, `SSOR`, `SOR`, `Cheby-SSOR` and **Iterative Samplers**: `SSOR-sampler`, `Cheby-acclerated Sampler`. For the detail implemtation, see `"../src/cheby.jl"` and `"cheby_sampler.jl"`
* Following plot shows the accuracy of Cheby-SSOR. 
* We calculate `xtrue` by `A \ b` from `julia`. And then plot the error between `xtrue` and our $\mathbf{x}^{(k)}_{\text{chebySSOR}}$, $\|x_{\text{true}} - \mathbf{x}^{(k)}\|_2$. For a detail implementaion, see `"results/graph-cheby-error.ipynb"`.
* Note that the error decreases in log-scale.

![](../results/graph_chebyssor_err.png)

* Followings are the `@benchmark` comparing between SSOR of `IterativeSolvers.jl` and Cheby-accelerated SSOR which we implemented.

In [6]:
using IterativeSolvers
include("../src/cheby_dw.jl")
A = laplacematrix(10);

In [7]:
b_ = copy(b)
@benchmark ssor(A, b_, 1.6641, maxiter=61300)  # ssor from IterativeSolvers.jl

BenchmarkTools.Trial: 
  memory estimate:  3.74 MiB
  allocs estimate:  245210
  --------------
  minimum time:     152.768 ms (0.00% GC)
  median time:      164.098 ms (0.00% GC)
  mean time:        163.219 ms (0.09% GC)
  maximum time:     172.918 ms (0.52% GC)
  --------------
  samples:          31
  evals/sample:     1

In [10]:
b_ = copy(b)
@time λ_max,λ_min = eigMm(A, 1.6641)
@benchmark k3_CB_ssor(A, b_, 1.6641, λ_max, λ_min, 640) # our implemented cheby-accelerated ssor solver.

  0.009268 seconds (541 allocations: 752.641 KiB)


BenchmarkTools.Trial: 
  memory estimate:  36.89 KiB
  allocs estimate:  33
  --------------
  minimum time:     1.478 ms (0.00% GC)
  median time:      1.716 ms (0.00% GC)
  mean time:        1.735 ms (0.18% GC)
  maximum time:     3.885 ms (58.21% GC)
  --------------
  samples:          2878
  evals/sample:     1

* We can see that implemented **Cheby-acceleraed SSOR** is about 15 times faster than *SSOR* of IterativeSolvers.jl including a time for calculating maximum and minmum eigen values of $M^{-1}A$.
* Also, we note that the memory efficiency of our accelerated SSOR.

#### 4. Actual calculation by Iterative Solvers
* In this section we reproduce *Table 3* in the paper.
* We solve $\mathbf{A}\mathbf{x} = \mathbf{b}$ where $\mathbf{A}$ is a $n=10$ matrix from the previous sextion and $\mathbf{b} \sim \mathbf{N}(\mathbf{0}, \mathbf{I}_{100})$ is a randomly generated for some fixed random seed.
* Following table is a summary of calculation results. Each solver was run until the residual became sufficiently small, $\|\mathbf{b} - \mathbf{A}\mathbf{x}^{(k)}\|_2 < 10^{-8}$.
* Reproduced results are given in `"results/count_floap.ipynb"`, `"results/count_iter.ipynb"` and included `.jl` codes in `"src/"` directory.


|Solver|$\omega$____|Number of iterations|Flops/iteration|Total Flops________|
|-------------|--------|:--------------------|------|--------------------|
|Richardson   | 1      |DNC                |1120 |-  |
|Jacobi       | -      |$6.92 \times 10^5$ |1320 |$9.13 \times 10^8$   |
|Gauss-Seidel | -      |$3.20 \times 10^5$ |1580 |$5.05 \times 10^8$   |
|SSOR         | 1.6641 |$6.13 \times 10^4$ |3240 |$1.99 \times 10^8$   |
|SOR          | 1.9852 |1668               |1580 |$2.64 \times 10^6$   |
|Cheby-SSOR   | 1      |1021               |3300 |$3.37 \times 10^6$   |
|Cheby-SSOR   | 1.6641 |636                |3300 |$2.10 \times 10^6$   |

* Following Table is a *Table 3* from the paper for the comparison with our results.

<center><img src="img/Table3paper.png"></center>

* $\mathcal{\rho}(\mathbf{M}^{-1}\mathbf{N})$ is a spectral radius of splitting matrices which implies a convergent speed.

#### 5. Empirical variances error plotting
* We reproduce the `Figure 2` in the paper.
* Relative error in covariance $\|\mathbf{A}^{-1} - \mathbf{S}_y^{(k)}\|_2/\|\mathbf{A}^{-1}\|_2$ versus number of iterations for a sampler implemented with `SSOR` and ω = 1, `SSOR` with optimal relaxation ω = 1.6641, and `SSOR with Chebyshev acceleration`. To numerically assess sampler convergence, the empirical sample covariance $\mathbf{S}_y^{(k)}$ was calculated for each iteration k using $10^4$ chains of samples.
* We reproduce a following figure by using our implementations. The figure shows a convergence of $\textbf{Var}(\mathbf{y}^{(k)}) \to \mathbf{A}^{-1}$. For the detail implementaion, see `"../results/graph-sampler.jl"`

<center><img src="img/figure2.png"></center>

* $y_1$ denotes Cheby-SSOR with w = 1 and $y_2$ denotes Cheby-SSOR with w = 1.6441. $y_3$ is SSOR with w = 1 and $y_4$ is SSOR with w = 1.6441. Finally, $y_5$ is non-iterative cholesky sampler of algorithm 1 in the paper. Below is the figure from the paper.

<center><img src="img/figure2paper.png"></center>

---
## 6. Real Data Analysis: Biofilm Image Gaussian Model

#### 0. Motivation
- In this section we deal with a A.Parker et al.'s JASA paper [Polynomial Accelerated Solutions to a Large Gaussian Model for Imaging Biofilms: In Theory and Finite Precision](https://doi.org/10.1080/01621459.2017.1409121). Because of hardness of the problem in our original paper, we choose the author's latter research, which includes our research.

- The object of this problem is sampling a $512 \times 512$ size biofilm image, which requires sampling from normal distribution with variance of a $512^2 \times 512^2$ matrix. The 6-2 example of the 2017 study even used a $10^6 \times 10^6$ matrix.

- The Chebyshev Accelerated Sampler needs to get an extreme eigenvalues as input variables. The problem is that attaining eigenvalues are demanding when the size of the matrix becomes larger. The 2017 study did not cover this, but mention that they obtained the extreme eigenvalues, $\lambda_{max}, \lambda_{min}$ using the cg-sampler. 

- Then how can we use the cg sampler to find the extreme eigenvalues? This requires to look at the series of studies done by Colin Fox and Albert Parker.

|Authors| Key of the paper |
|-------------------------------|-------------------------------------------------------------------|
| Fox, C., and Parker, A.(2012) | Introduced a conjugate gradient sampler which then could inexpensively estimate some of the eigenvalues of $A$.|
| Fox, C., and Parker, A.(2014) | First suggested a Cheby-accelerated sampler.|
| __Fox, C., and Parker, A.(2017)__ | Proved convergence in distribution for Gibbs samplers corresponding to _any_ matrix splitting and accelerated by _any_ polynomial that is independent of the Gibbs iteration.|
| Parker, A. et al.(2018) | Combined all this methods and proposed PCG and __PCG-Chebyshev Accelerated Sampler__.|


* Solving linear inverse problem for large matrices requires to obtain an extreme eigen values of $M_{SSOR}^{-1}A$.

* The most important parts are well documented in the 2018 paper. The crucial fact that PCG sampler accoplishes are
    - The PCG sampler with preconditiner equal to the splitting matrix $M = M_1M_1^T$ provides an avenue to estimate the extreme eigenvalues of $M^{-1}A$ that are required by Chebyshev. A $k \times k$ tridiagonal matrix is built from the PCG parameters and the eigenvalues of this tridiagonal, found at a negligible $k^2$ flops when $k<<n$, are the required extreme eigenvalues of $M^{-1}A $. (2018, p1437)
    - PCG provides an estimate of the mean $\mu = A^{-1}b \neq 0$.The PCG sampler is used to perform the mean calculation because PCG is a faster linear solver than Chebyshev and will find μ after a finite number of iterations. Put another way, the Chebyshev sampler can sample from $N(0,A−1)$ much faster compared to sampling from the Normal with nonzero mean.
    
* So far, in order to adapt our sampler to a larger size problems, we implemented the followings:

|Paper| What we implemented |
|-------------------------------|-------------------------------------------------------------------|
| Fox, C., and Parker, A.(2012) | CG Accelerated Sampler (_pcg_sampler.jl_)|
| Fox, C., and Parker, A.(2014) | Chebyshev-accelerated sampler(_cheby_sampler.jl_)|
| __Fox, C., and Parker, A.(2017)__ | Chebyshev-accelerated sampler(_cheby_sampler.jl_)|
| Parker, A. et al.(2018) | PCG Accelerated Sampler, PCG-Chebyshev Accelerated Sampler(_pcg_cheby.jl_)|
| Additional |Generating Lanczos matrix(lemma 2.1 in paper 2012, _pcg_cheby.jl_), Generating generalized lattice sparse precision matrix( EX 6.2 in paper 2017 and EX in 2018 , _matrixgen.jl_)|



#### 1. Data setting

- Confocal microscopes (CM) capture a set of planar “slices” or images, parallel to, and at different distances from, the bottom of the biofilm
  where it is attached to a surface. In this paper, they analyze a sequence of CM images over time of a green fluorescing *Staphylococcus aureus* biofilm grown under controlled conditions.

- The data set is given by a video file(see [supplementary material](https://www.tandfonline.com/doi/suppl/10.1080/01621459.2017.1409121?scroll=top)). In the video file, we use a first 40 frames of biofilm images about 10 minute. Following image is a 1st frame of the video file.

<img src="img/frame_1.png" width="500" height="600" />

- Biofilm captures $620 \mu m \times 620 \mu m$ planar with a vertical range of $112 \mu m$. Vertical value means the observed height of Staphylococcus aureus at (x, y) pixel point. The 3D pixelation is $512 \times 512 \times 17$ pixels with a $512 \times 512$ pixel representation for each planar slice. (즉, 박테리아 군집을 높이에 따라 17개의 slicing 단면을 촬영한 정보를 한 frame으로 합쳐놓았다고 생각하면 된다.) After resizing, it becomes like as following.

<img src="img/gframe_1.png" width="300" height="300" />

#### 2. Modeling
- Note that we can consider a height information as a discretize observed surface representation. We want to recover the whole surface information $\theta$.

- From the observed The linear statistical model that we apply to the surface profile is
  $$y = F\theta + \epsilon.$$
  
- The random vector $y \in \mathbb{R}^{512 \times 512}$ representation of the biofilm surface calculated from the CM image. $\theta$ is the true biofilm surface that we want to estimate. $F$ is a blurring of the surface, $\epsilon \sim N(0, \Sigma_y)$. The likelihood is $\pi(y|\theta, \Sigma_y) = N(F\theta, \Sigma_y)$. Let's introduce the prior $\pi(\theta) = N(0, \frac{1}{\lambda}W^{-1})$ with an unknow parameter $\lambda$ from the smoothness assumption. $W$ is a $512^2 \times 512^2$ precision matrix defined from the Section 6.1 $\mathbf{A}$ with $n = 512$. 

- After some Bayesian perspective calculation procedures, it reduces to sample from
  $$
  \pi(\theta|y, \sigma^2, \lambda) = N(\frac{1}{\sigma^2}A^{-1}y, A^{-1})
  $$
  with precision matrix $A = \frac{1}{\sigma^2}I + \lambda W$ and $1/\sigma^2, \lambda \sim Gamma(\alpha = 1, \beta = 10^{-4})$.

#### 3. Implementation

- 2018 JASA paper uses iterative samplers from large multivariate Gaussians to reconstruct bioflim surfaces and to estimate the effect of the treatment on the biofim's volume. Here, we will use CSLM images to reconstruct biofilm surfaces using the accelerated chebyshev-ssor sampler we implemented. 
 
##### The need for PCG - Chebyshev Accelerated Sampler
- Finding eigenvalues is not easy for large dimensional matrices. Since our implementation of chebyshev-ssor requires extreme eigenvalues, $\lambda_{max}, \lambda_{min}$, for $512 ^ 2 \times 512 ^ 2$ sized matrix, this computation is very demanding. `Arpack.jl`'s `eigs` fails to calculate a extreme eigen value of these matrices for high-dimensional case.
- The PCG-Chebyshev algorithm proposed in the 2018 JASA paper that makes this problem easier. We can create a Lanczos tridiagonal matrix using a set of values, $\{\gamma_k, \beta_k\}$, attained during PCG sampling and it is known that the extreme eigenvalues of $M^{-1}A$ that are required by Chebyshev are the extreme eigenvalues of this $k \times k$(here, $k$ is the number of iterations) tridiagonal matrix.



We implemented all the samplers, but found that the kernel dies in running all at once using pcg_cheby function. Instead, the results could be obtained by sequentially applying pcg sampler-> eigenvalue extraction-> Chebyshev sampler. To see the process, see "data_run.ipynb"

The paper procured samples by perfoming _$10^4$_ iterations of sampling and then averaging them for each fame in the video. This took about 2.5 hours of running on each frame for them. Running out of time, we instead only sampled _once_ and produced the result, which would generate a large variation. 

Further, we used the minimum value of the frame picture that we used as the reference as the scale value, and as changing this, the values in the graph can change.

* sample once (our result)

<img src="img/run_1_times.png" width="500" height="500" />

* sample 10000th and averages (from paper)|

<img src="img/result_from_paper.png" width = "500" height="500"/>


We believe further research will prove that our sampler is effective for large sampling problems.

---
## Refrences
- Fox Colin, and Albert Parker. “Accelerated Gibbs sampling of normal distributions using matrix splittings and polynomials.” Bernoulli 23.4B (2017): 3711-3743.
- Albert Parker et al. "Polynomial Accelerated Solutions to a Large Gaussian Model for Imaging Biofilms: In Theory and Finite Precision.", Journal of the American Statistical Association Volume 113, 2018 - Issue 524.
- Owe Axelsson, Iterative Solution Methods, Cambridge, 1994.
- IterativeSolver.jl, `julia` package, https://github.com/JuliaMath/IterativeSolvers.jl.